# LangChain, PostgreSQL, Gemini API でドキュメント QA を行う例

以下のパッケージをインストールします。
- LangChain 関係のパッケージ
- PDF の扱いに必要なパッケージ
- CloudSQL の PostgreSQL インスタンスにアクセスするためのパッケージ

In [ ]:
!pip install --user \
  langchain==0.3.27 transformers==4.36.0 \
  pypdf==6.0.0 cryptography==42.0.4 \
  pg8000==1.30.4 cloud-sql-python-connector[pg8000]==1.7.0 \
  langchain-google-vertexai==2.0.28 \
  google-cloud-aiplatform==1.111.0 \
  langchain-community==0.3.29

**注意：次のセルを実行する前にカーネルをリスタートしてください。**

エンベディングモデル `gemini-embedding-001` を利用するための補助関数を定義します。

In [1]:
import vertexai
from google import genai

vertexai.init(location='asia-northeast1')
[PROJECT_ID] = !gcloud config list --format 'value(core.project)'

client = genai.Client(vertexai=True, project=PROJECT_ID, location='us-central1')

def embed_documents(page_contents):
    result = client.models.embed_content(
        model='gemini-embedding-001',
        config=genai.types.EmbedContentConfig(
            output_dimensionality=768,
            task_type='RETRIEVAL_DOCUMENT'
        ),
        contents= [page_contents])
    return [item.values for item in result.embeddings]

def embed_query(query):
    result = client.models.embed_content(
        model='gemini-embedding-001',
        config=genai.types.EmbedContentConfig(
            output_dimensionality=768,
            task_type='QUESTION_ANSWERING'
        ),
        contents= [query])
    return result.embeddings[0].values

関数 `embed_documents()` は検索対象のドキュメントに対する埋め込みベクトルを生成します。1つのテキストから、768次元の埋め込みベクトルが得られます。

In [2]:
embedding_vectors = embed_documents(['今日は快晴です'])

In [3]:
len(embedding_vectors), len(embedding_vectors[0])

(1, 768)

In [4]:
embedding_vectors[0][:5]

[-0.02324664406478405,
 0.0024090830702334642,
 0.0015183135401457548,
 -0.09230884164571762,
 -0.004308043979108334]

デジタル庁が一般公開している「[アジャイル開発実践ガイドブック](https://www.digital.go.jp/resources/standard_guidelines)」の PDF ファイルをダウンロードします。

In [5]:
base_url = 'https://raw.githubusercontent.com/google-cloud-japan/sa-ml-workshop/main'
!wget -q $base_url/genAI_book/PDF/agile-guidebook.pdf

PDF ファイルの内容をページごとに分割して、それぞれのページの埋め込みベクトルを生成します。

In [6]:
from langchain_community.document_loaders import PyPDFLoader
pages = PyPDFLoader('agile-guidebook.pdf').load()
page_contents = [page.page_content for page in pages]
embedding_vectors = embed_documents(page_contents)

In [7]:
len(embedding_vectors), len(embedding_vectors[0])

(37, 768)

PostgreSQL に接続するためのコネクションプールを用意します。

In [8]:
import google.auth
import sqlalchemy
from google.cloud.sql.connector import Connector

_, project_id = google.auth.default()
region = 'asia-northeast1'
instance_name = 'genai-app-db'
INSTANCE_CONNECTION_NAME = '{}:{}:{}'.format(
    project_id, region, instance_name)
DB_USER = 'db-admin'
DB_PASS = 'genai-db-admin'
DB_NAME = 'docs_db'

connector = Connector()

def getconn():
    return connector.connect(
        INSTANCE_CONNECTION_NAME, 'pg8000',
        user=DB_USER, password=DB_PASS, db=DB_NAME)

pool = sqlalchemy.create_engine('postgresql+pg8000://', creator=getconn)

埋め込みベクトルをデータベースから削除、および、データベースに保存する関数を定義します。

In [9]:
def delete_doc(docid):
    with pool.connect() as db_conn:
        delete_stmt = sqlalchemy.text(
            'DELETE FROM docs_embeddings WHERE docid=:docid;'
        )
        parameters = {'docid': docid}
        db_conn.execute(delete_stmt, parameters=parameters)
        db_conn.commit()

def insert_doc(docid, uid, filename, page, content, embedding_vector):
    with pool.connect() as db_conn:
        insert_stmt = sqlalchemy.text(
            'INSERT INTO docs_embeddings \
             (docid, uid, filename, page, content, embedding) \
             VALUES (:docid, :uid, :filename, :page, :content, :embedding);'
        )
        parameters = {
            'docid': docid,
            'uid': uid,
            'filename': filename,
            'page': page,
            'content': content,
            'embedding': embedding_vector
        }
        db_conn.execute(insert_stmt, parameters=parameters)
        db_conn.commit()

先ほどのドキュメントの各ページの埋め込みベクトルをデータベースに保存します。

In [10]:
docid = 'dummy_id'
uid = 'dummy_uid'
filename = 'agile-guidebook.pdf'

delete_doc(docid)        
for c, embedding_vector in enumerate(embedding_vectors):
    page = c+1
    insert_doc(docid, uid, filename, page,
               page_contents[c], str(embedding_vector))

関数 `embed_query` は質問文に対する埋め込みベクトルを生成します。得られた埋め込みベクトルから関連性の高いページ（埋め込みベクトルの値が近い）のトップ3を取得します。

In [11]:
question = 'アジャイル開発の採用に慎重になるべきケースはありますか？'
question_embedding = embed_query(question)

with pool.connect() as db_conn:
    search_stmt = sqlalchemy.text(
        'SELECT filename, page, content, \
                1 - (embedding <=> :question) AS similarity \
         FROM docs_embeddings \
         WHERE uid=:uid \
         ORDER BY similarity DESC LIMIT 3;'
    )
    parameters = {'uid': uid, 'question': str(question_embedding)}
    results = db_conn.execute(search_stmt, parameters=parameters)

text = ''
source = []
for filename, page, content, _ in results:
    source.append({'filename': filename, 'page': page})
    text += content + '\n'

In [12]:
source

[{'filename': 'agile-guidebook.pdf', 'page': 16},
 {'filename': 'agile-guidebook.pdf', 'page': 19},
 {'filename': 'agile-guidebook.pdf', 'page': 18}]

得られたページのテキストに基づいて、質問の回答を生成します。

In [13]:
from langchain_google_vertexai import VertexAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain

llm = VertexAI(model_name='gemini-2.5-flash-lite', location='us-central1',
               temperature=0.1, max_output_tokens=256)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=6000, chunk_overlap=200)
qa_chain = load_qa_chain(llm, chain_type='refine')
qa_document_chain = AnalyzeDocumentChain(
    combine_docs_chain=qa_chain, text_splitter=text_splitter)

prompt = '{} 日本語で3文程度にまとめて教えてください。'.format(question)
answer = qa_document_chain.invoke({'input_document': text, 'question': prompt})
print(answer['output_text'])

/var/tmp/ipykernel_57376/2429631627.py:10: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm, chain_type='refine')
/var/tmp/ipykernel_57376/2429631627.py:11: LangChainDeprecationWarning: The class `AnalyzeDocumentChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~example in API reference with more detail: https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.base.A

はい、アジャイル開発の採用に慎重になるべきケースがあります。

具体的には、大規模な情報システム、業務内容が極めて複雑な場合、またはミッションクリティカルな（障害や誤作動が許されない）業務においては、慎重な判断が必要です。これらのケースでは、どこまでを事前に詳細化し、どの部分をアジャイルで開発するか、そして品質をどのように確保・向上させるかといった検討が不可欠となります。
